In [1]:
import os
import random
import rasterio
import numpy as np
import geopandas as gpd
from osgeo import gdal
from PIL import Image

## This script is used to generate training data for training networks that can help us automatically adjust geojson points' locations. 

#There are mannually adjusted geojson files under ./geojson/adjusted/ folder. By randomly change the points of those adjusted points, we can simulate the situations that nodes are not correctly placed. The input image of the network to be trained is the cropped images under ./cropped/Train/ folder, they are images with center changed randomly.  X/Y change range: (-80,80) (random: normal distribution, avg=0, sd=10)

In [2]:
# Define a function to crop an image
def crop_image(img, px, py, crop_size=150):
    half_size = crop_size // 2
    start_x = max(px - half_size, 0)
    end_x = min(px + half_size, img.shape[1])
    start_y = max(py - half_size, 0)
    end_y = min(py + half_size, img.shape[0])
    
    cropped_image = img[start_y:end_y, start_x:end_x]
    return cropped_image


def save_as_png(array, path):
    img = Image.fromarray(array)
    img.save(path)
    
    
def read_RGB_from_geotif(src):  
    band_count = src.count
    if band_count >= 8:
        r = src.read(5)
        g = src.read(3)
        b = src.read(2)
    elif band_count <= 4:
        r = src.read(1)
        g = src.read(2)
        b = src.read(3)
    img = np.stack((r, g, b), axis=-1)
    img_uint8 = (img / img.max() * 255).astype(np.uint8)
    return img_uint8
    
    
def latlon_2_imxy(points,bounds,image_size):
    min_x = bounds[0]
    min_y = bounds[1]
    len_x = bounds[2] - bounds[0]
    len_y = bounds[3] - bounds[1]
    xy = np.zeros((len(points),2))
    xy = np.array([[p.x, p.y] for p in points])      
    xy[:, 0] = image_size[0] * ((xy[:, 0] - min_x) / len_x)
    xy[:, 1] = image_size[1] - (image_size[1] * ((xy[:, 1] - min_y) / len_y))
    xy = np.floor(xy)
    return xy  

In [3]:
# Define Path
geojson_dir = './geojson/adjusted_base_center/'
tif_dir = '/media/HD_Matcha_2/Yan/all_train_val/'
ground_truth_dir = './cropped/Ground_Truth/'
train_dir = './cropped/Train/'

# Create output folders
os.makedirs(ground_truth_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)

# Loop all geojson files
for geojson_file in os.listdir(geojson_dir):
    if geojson_file.endswith('.geojson'):
        geojson_path = os.path.join(geojson_dir, geojson_file)
        print(geojson_path)
        gdf = gpd.read_file(geojson_path)
        points = gdf["geometry"]
        print(points)
        tif_file = os.path.join(tif_dir, geojson_file.replace('.geojson', '.tif'))
        tiff_image = rasterio.open(tif_file)        
        bounds = tiff_image.bounds    
        if "metre" in tiff_image.crs.to_string():
            print("The Unit is metre. coordinate are transfered into lat-long in degrees.")
            bounds = transform_bounds(tiff_image.crs, "WGS84",
                                             bounds.left, bounds.bottom, bounds.right, bounds.top)
        else:
            print("The Unit is degree.")        
        image_points_xy = latlon_2_imxy(points,bounds,[tiff_image.width,tiff_image.height])        
        img_rgb = read_RGB_from_geotif(tiff_image)
        for i, point in enumerate(points):
            input_point = np.array([[image_points_xy[i][0],image_points_xy[i][1]]])
            x, y = int(input_point[0][0]), int(input_point[0][1])   
            # crop Ground Truth image
            cropped_image = crop_image(img_rgb, x, y)
            ground_truth_path = os.path.join(ground_truth_dir, f"{geojson_file.replace('.geojson', '')}_{i}.png")
            save_as_png(cropped_image, ground_truth_path)
            
            # Randomly move points and crop images for training
            for _ in range(30):  # Generate 10 random cropped images for each point
                dx = int(np.clip(np.random.normal(0, 10), -80, 80))
                dy = int(np.clip(np.random.normal(0, 10), -80, 80))
                
                moved_x = x + dx
                moved_y = y + dy
                
                cropped_image = crop_image(img_rgb, moved_x, moved_y)
                
                x_direction = f"P{dx}" if dx > 0 else f"N{-dx}"
                y_direction = f"P{dy}" if dy > 0 else f"N{-dy}"
                
                train_path = os.path.join(train_dir, f"{geojson_file.replace('.geojson', '')}_{i}_{x_direction}_{y_direction}.png")
                save_as_png(cropped_image, train_path)

print("Cropping complete.")

./geojson/adjusted_base_center/1909.geojson
0    POINT (106.75897 10.82453)
1    POINT (106.75903 10.82038)
2    POINT (106.76003 10.82693)
3    POINT (106.75960 10.82642)
4    POINT (106.75850 10.82853)
Name: geometry, dtype: geometry
The Unit is degree.


ERROR 1: PROJ: proj_identify: /home/irene-oceanus/anaconda3/envs/yolo/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


./geojson/adjusted_base_center/1921.geojson
0    POINT (106.76157 10.82935)
1    POINT (106.76112 10.83008)
2    POINT (106.76361 10.82919)
3    POINT (106.76299 10.83239)
4    POINT (106.76564 10.83273)
5    POINT (106.76140 10.83231)
6    POINT (106.76083 10.83266)
7    POINT (106.76289 10.83447)
8    POINT (106.76242 10.83658)
Name: geometry, dtype: geometry
The Unit is degree.


ERROR 1: PROJ: proj_identify: /home/irene-oceanus/anaconda3/envs/yolo/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


./geojson/adjusted_base_center/1963.geojson
0    POINT (32.91814 39.87668)
1    POINT (32.91742 39.87956)
2    POINT (32.91701 39.87308)
3    POINT (32.91660 39.87706)
Name: geometry, dtype: geometry
The Unit is degree.


...

./geojson/adjusted_base_center/2036.geojson
0    POINT (30.40785 50.50230)
1    POINT (30.40516 50.50466)
2    POINT (30.40320 50.50637)
3    POINT (30.40904 50.50127)
4    POINT (30.41361 50.50120)
5    POINT (30.41356 50.50102)
6    POINT (30.41155 50.50004)
7    POINT (30.41119 50.50013)
Name: geometry, dtype: geometry
The Unit is degree.


ERROR 1: PROJ: proj_identify: /home/irene-oceanus/anaconda3/envs/yolo/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


./geojson/adjusted_base_center/1602.geojson
0    POINT (-71.22303 -29.73049)
Name: geometry, dtype: geometry
The Unit is degree.


ERROR 1: PROJ: proj_identify: /home/irene-oceanus/anaconda3/envs/yolo/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


Cropping complete.
